In [13]:
import cv2
import os
import mediapipe as mp 
import numpy as np 
import csv
import sys

In [14]:
root_dir = 'D:/dataset/MELD.Raw/MELD subfolder'

In [3]:
labels = []

# Define a dictionary to map the folder names to labels
label_dict = {'anger' : 'anger', 'disgust' : 'disgust', 'fear': 'fear', 'joy' : 'joy', 'neutral': 'neutral', 'sadness' : 'sadness', 'surprise': 'surprise'}

In [4]:
coords_num = 501 # the number of landmarks detected by Mediapipe for the face (468) and the pose (33) 
landmarks = ['class']
for value in range(1, coords_num+1):
    landmarks += ['x{}'.format(value), 'y{}'.format(value),'z{}'.format(value), 'v{}'.format(value)]

In [15]:
with open('D:/dataset/MELD.csv', mode='w', newline='') as coords:
    csv_writer = csv.writer(coords, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

# Detections

In [17]:
# initializing the landmarks drawing utilites 
mp_drawing = mp.solutions.drawing_utils
# mediapipe landmark detection models
mp_holistic = mp.solutions.holistic
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Traverse the directory tree
    for subdir, dirs, files in os.walk(root_dir):
        # Extract the label from the folder name
        label = os.path.basename(subdir)
        # Check if the label is in the dictionary
        if label in label_dict.values():
            # Iterate over the files in the current directory
            for file in files:
                # Check if the file is a video
                if file.endswith('.MP4'):
                    # Construct the full file path
                    file_path = os.path.join(subdir, file)
                    # Read the video using OpenCV
                    video_capture = cv2.VideoCapture(file_path)
                    # Check if the video was opened successfully
                    if not video_capture.isOpened():
                        print("Error opening video")
                        sys.exit()
                    # Loop over the frames in the video
                    while True:
                        # Read a frame from the video
                        ret, frame = video_capture.read()
                        # Check if we reached the end of the video
                        if not ret:
                            break
                        # Pass the frame to medipipe for detection
                        results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                        # Draw face landmarks
                        mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                                  mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                                  mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                                  )
                        # Draw right hand landmarks
                        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                                  mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                                  )
                        # Draw left hand landmarks
                        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                                  )
                        
                        # Draw left hand landmarks
                        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                              )

                         # Export Coordinates
                        try:
                            # Extract pose landmarks
                            pose = results.pose_landmarks.landmark
                            pose_row= list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]for landmark in pose]).flatten())

                            # Extract face landmarks
                            face = results.face_landmarks.landmark
                            face_row= list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]for landmark in face]).flatten())

                            # concatenate rows
                            row = pose_row + face_row

                            # Append class name
                            row.insert(0, label_dict[label])

                            # Export to CSV
                            with open('D:/dataset/MELD.csv', mode='a', newline='') as coords:
                                csv_writer = csv.writer(coords, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                                csv_writer.writerow(row) 
                        except:
                            pass

Error opening video


SystemExit: 

C:\Users\toto1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# -

In [11]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
cap = cv2.VideoCapture(r'C:\Users\loading emotic dataset\DR\surprise\dia38_utt11.MP4')
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:

            # Recolor Feed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False        

            # Make Detections
            results = holistic.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # 1. Draw face landmarks
            mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                     mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                     mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                     )

            # 2. Right hand
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                     )

            # 3. Left Hand
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                     )

            # 4. Pose Detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )

            cv2.imshow('Raw Webcam Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    
        else:
            break
cap.release()
cv2.destroyAllWindows()